In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
import time

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Use the appropriate model variant (e.g., yolov8s.pt for higher accuracy)



In [3]:
# Define traffic signal states
signal_states = ["NORTH-SOUTH", "EAST-WEST"]

# Initialize variables
signal_duration = 10  # seconds for each signal state
vehicle_classes = [2, 3, 5, 7]  # YOLO class IDs for car, motorcycle, bus, truck




In [4]:
def process_frame(frame, model):
   
    height, width, _ = frame.shape
    
    # Define regions for each side of the intersection
    regions = {
        "NORTH": [(0, 0), (width, height // 4)],
        "SOUTH": [(0, 3 * height // 4), (width, height)],
        "EAST": [(3 * width // 4, 0), (width, height)],
        "WEST": [(0, 0), (width // 4, height)],
    }
    
    vehicle_counts = {side: 0 for side in regions}
    
    # Perform object detection
    results = model(frame, verbose=False)
    detections = results[0].boxes.boxes.cpu().numpy()  # Extract detections as numpy array
    
    for detection in detections:
        x1, y1, x2, y2, confidence, class_id = detection
        if int(class_id) in vehicle_classes:
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2  # Calculate center of the bounding box
            
            # Determine which region the vehicle is in
            for side, ((x_start, y_start), (x_end, y_end)) in regions.items():
                if x_start <= cx <= x_end and y_start <= cy <= y_end:
                    vehicle_counts[side] += 1
                    break

    return vehicle_counts



In [5]:
def decide_signal(vehicle_counts):
   
    north_south = vehicle_counts["NORTH"] + vehicle_counts["SOUTH"]
    east_west = vehicle_counts["EAST"] + vehicle_counts["WEST"]
    
    if north_south >= east_west:
        return "NORTH-SOUTH"
    else:
        return "EAST-WEST"




In [6]:
def main(video_path):
    # Open video capture
    cap = cv2.VideoCapture(video_path)
    current_signal = "NORTH-SOUTH"
    last_switch_time = time.time()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (640, 640))  # Resize to 640x640 for faster processing
        # Resize frame for consistent processing
        frame = cv2.resize(frame, (640, 640))
        
        # Process the frame to count vehicles
        vehicle_counts = process_frame(frame, model)
        print(f"Vehicle counts: {vehicle_counts}")
        
        # Decide signal state based on vehicle counts
        if time.time() - last_switch_time > signal_duration:
            current_signal = decide_signal(vehicle_counts)
            last_switch_time = time.time()
        
        # Display current signal state
        cv2.putText(frame, f"Current Signal: {current_signal}", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Show the frame
        cv2.imshow("Traffic Control", frame)
        
        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Run the program


In [ ]:
if __name__ == "__main__":
    video_path = r"C:\Users\ashri\Documents\Ash\DSU\SEM 5\AIML\MINI project\video1.mp4"
    main(video_path)